# Die `for`-Schleife

## Was behandeln wir in diesem Notebook / Lernziele

<div class="alert alert-success">

<b>Fragestellung:</b>
<ul>
    <li> Wie kann ich existierende <code>Unix</code>-Programme kombinieren, um neue und komplexere Aufgaben zu erledigen.</li>
</ul>

<b>Aspekte der Fragestellung:</b>
<ul>
    <li> Reproduzierbare Automatisierung von Aufgaben durch Schleifen</li>
</ul>

<b> Zeitaufwand für diese Lektion: </b> 
<ul>
    <li> Durcharbeiten des Textes: 30 min</li>
    <li> Verständnisfragen: 20 min</li>
</ul>
</div>

## Schleifen
Schleifen sind ein wichtiges Konstrukt der Programmierung. Sie erlauben es uns, Befehlssequenzen automatisch und reproduzierbar auf jedes Element *einer Liste* anzuwenden. Neben den in der letzten Lektion besprochenen Pipelines sind sie der zweite wichtige Baustein, um `Unix`-Kommandos für komplexe Aufgaben zu kombinieren. Sie sind essentiell, um langweilige und sich wiederholende Aufgaben fehlerfrei, effektiv und mit einem Mindestmaß an manueller Intervention durchzuführen.

## Die `for`-Schleife
Die `bash`-Shell bietet mehrere Schleifenkonstrukte und wir werden uns in diesem Tutorial auf die so-genannte `for`-Schleife beschränken.

Annika arbeite noch an ihrem Seminar zu den Monden der Planeten unseres Sonnensystems.

In [ ]:
cd ~/Seminar/Planeten_Daten

Sie möchte für jeden Planeten den Mond mit dem geringsten Abstand zum Planeten ermitteln. Mit der Struktur der Planetendateien (siehe [hier](04_Shell_Pipelines_und_Filter.ipynb#Planetendatei_Struktur)) ist dies der Mond mit dem niedrigsten Wert in der vierten Spalte. Um diesen für Jupiter zu erhalten, verwendet Annika die Pipeline

In [ ]:
sort -g -k 4 Jupiter.dat | head -n 1  # zeige den Mond mit dem geringsten Abstand zu Uranus

Die Option `-k 4` für `sort` sorgt dafür, dass *`sort `die ersten drei Spalten nicht verwendet und erst der Inhalt ab Spalte vier für die Sortierung herangezogen wird*. Dies ist im vorliegenden Fall effektiv eine Sortierung nach dem Mondabstand vom Planeten.

Um diese Aufgabe für jeden Planeten zu lösen, muss obige Pipeline jedes mal mit der entsprechenden Planetendatei neu ausgeführt werden. Das Problem, exakt dieselbe Befehlssequenz mit einer *Variablen* (hier die Planetendateien) wiederholt ausführen zu müssen, ist ein typischer Anwendungsfall für eine Schleife.

Die allgemine Struktur der `for`-Schleife ist wie folgt:
```bash
for VARIABLE in LISTE
do
  BEFEHLSSEQUENZ mit ${VARIABLE}
done
```
Eine genauere Erklärung folgt weiter unten.

In unserem konkreten Fall sieht das, zunächst für die zwei Planeten Jupiter und Saturn, wie folgt aus

In [ ]:
# Schleife, um den Befehl sort -g -k 4 ..... | head -n 1
# auf die 'Liste' Uranus.dat und Saturn.dat anzuwenden
#
for DATEI in Jupiter.dat Saturn.dat
do
  sort -g -k 4 ${DATEI} | head -n 1
done  

Das Schlüsselwort `for` sagt der Shell, dass eine Befehlssequenz für jedes Element einer *Liste* zu wiederholen ist. In jedem Schleifendurchlauf (auch Iteration genannt) wird ein Element der Liste einer *Schleifenvariable* zugeordnet und die Befehle innerhalb der Schleife werden ausgeführt, bevor zum nächsten Listenelement gesprungen wird. Innerhalb der Schleife können wir auf den *Wert* der *Schleifenvariable* mit dem Konstrukt `${...}` zugreifen, wobei `...` für den *Variablennamen* stehen.

Im konkreten Fall hat die Liste zwei Elemente, die Dateinamen `Jupiter.dat` und `Saturn.dat`. In jedem der zwei Schleifendurchläufe wird ein Dateiname der Schleifenvariable `DATEI` zugewiesen und der `sort`-Befehl ausgeführt. In der ersten Iteration hat die Variable `DATEI` den *Wert* `Jupiter.dat`, auf welchen mit `${DATEI}` zugegriffen wird. Es wird also in der ersten Iteration  der Befehl `sort -g -k 4 Jupiter.dat | head -n 1` ausgeführt. Das Ganze wiederholt sich in der zweiten Iteration, wobei der Wert der Variable `DATEI` jetzt `Saturn.dat` ist. Da die Liste im gegebenem Fall nur zwei Elemente hat, endet die Schleife nach zwei Iterationen.

Der allgemeine Programmfluss einer Schleife und die konkreten Schleifenelemente des aktuellen Beispiels sind noch einmal in folgender Figur dargestellt.

<img src="figuren/Shell_for_Schleife_fig1_und_2.png?modified=12345678" style="width: 600px;">

## Einschub: Verschiedenes im Zusammenhang mit der  `for`-Schleife

### Hilfe bei der Schleifenkonstruktion

Anfängern fällt es oft schwer, Schleifen korrekt aufzubauen und damit verbundene Fehler sind nach Schleifendurchführung manchmal schwer zu finden. Eine Hilfe ist es, sich vor der aktuellen Schleifenausführung die Schleifenbefehle anzeigen zu lassen und sich so zu vergewissen, dass sie *korrekt* sind. Hierbei hilft das Kommando `echo`, welches nichts weiter tut, als in Anführungsstrichen eingebetteten Text auf dem Bildschirm auszugeben.

In [ ]:
echo "Annika Oliver Thomas"  # gibt einfach 'Annika Oliver Thomas' aus

Interessant ist dieser Befehl in Zusammenhang mit Schleifen, da Variablen *vor der Ausgabe* ausgewertet werden. 

In [ ]:
# Die Schleife von oben. Der Schleifenbefehl ist aber in
# einen echo-Befehl eingebettet. Er wird dadurch angezeigt, aber
# nicht ausgeführt.
#
for DATEI in Jupiter.dat Saturn.dat
do
  echo "sort -g -k 4 ${DATEI} | head -n 1"
done  

Mit diesem Trick kann man sich die Befehle, die die Schleife ausführen würde, erst einmal ansehen, *bevor* sie abgearbeitet werden. Wenn alles in Ordnung ist, entfernt man den `echo`-Befehl samt Anführungszeichen und führt die Schleife aus.

### Sukzessive Konstruktion einer Textdatei mit einer Schleife

Annika möchte sich die Monde mit dem geringsten Abstand zu den Planeten nicht nur ausgeben lassen, sondern diese in eine Datei `nahe_Planeten.txt` speichern. Um dies zu erreichen, gibt es eine weitere Form der Ausgabeumlenkung, die wir [in der vierten Lektion](04_Shell_Pipelines_und_Filter.ipynb#Ausgabeumlenkung) eingeführt haben. Die Ausgabeumlenkung leitet die Ausgabe eines Programms in eine Textdatei um. Bei Existenz dieser Datei, wird diese allerdings *überschrieben*.

In [ ]:
echo "Annika" > test.txt   # lenke Annika in test.txt
echo "Oliver" > test.txt   # lenkt Oliver in test.txt;
                           # Annika wird überschrieben.
cat test.txt                           

Es ist also mit dem `>`-Operator nicht möglich, eine Datei schrittweise aufzubauen und zu erweitern. Mit dem zu `>` verwandten Operator `>>` wird ebenfalls die Ausgabe eines Befehls in eine Datei umgelenkt. Falls die Ausgabedatei noch nicht existiert, so haben beide Operatoren denselben Effekt. Existiert die Datei jedoch, so wird die Ausgabe mit `>>` *an die bestehende Datei angehängt*! 

In [ ]:
echo "Annika" > test.txt   # lenke Annika in test.txt
echo "Oliver" >> test.txt   # hängt Oliver an test.txt an
cat test.txt                           

Hiermit kann Annika ihre Datei `nahe_Planeten.txt` erstellen.

In [ ]:
# Die nahem Monde werden alle in eine Datei
# geschrieben.
#
for DATEI in Jupiter.dat Saturn.dat
do
  sort -g -k 4 ${DATEI} | head -n 1 >> nahe_Planeten.txt
done 
cat nahe_Planeten.txt

### Variablennamen

Wir haben in unserem Beispiel für den Variablennamen der Schleife `DATEI` gewählt, da es ishcb ei den Listenelementen um Dateinamen handelt. Es ist jedoch für die Funktionsweise der Schleife egal, welchen Namen wir wählen. So wäre unsere Schleife von eben äquivalent zu folgender Variante

In [ ]:
for x in Jupiter.dat Saturn.dat
do
  sort -g -k 4 ${x} | head -n 1
done 

Aus Gründen der Lesbarkeit sollte jedoch ein dem Problem angepasster Name gewählt werden. Das hilft Ihnen und anderen, Ihre Schleifen besser und einfacher zu verstehen - vor allem, wenn Sie diese nach längerer Zeit wieder bearbeiten und an ein neues Problem anpassen müssen. Obwohl es nicht zwingend erforderlich ist, hat es sich auch eingebürgert, für Schleifenvariablen nur Großbuchstaben zu verwenden. Hiermit sind sie von anderen Schleifenelementen wie Kommandonamen oder Optionen sofort zu unterscheiden.

### Schleifen in der Shell

## Annikas Bachelor-Arbeit: Statistiken aller Beobachtungen

<div class="alert alert-success">
<b>Zum Mitnehmen</b>
<ul>
    <li> <code>befehl >> datei</code> hängt die Ausgabe von <code>befehl</code> an <code>datei</code> an.</li>
    <li> <code>echo "text"</code> Gibt <code>text</code> auf dem Bildschirm aus.</li>
    <li> Die <code>for</code>-Schleife kann eine Befehlssequenz auf alle Elemente einer Liste ausführen.</li>
</ul>
</div>